# Perform A/B testing with a new version of a model

## Train another version of your XGBoost model

This lab has a dependency on first completing the base XGBoost lab. To get started with this lab, locate your xgboost training job. Navigate to the `Training jobs` section of the SageMaker console. Find your XGBoost training job and select it. Click `Clone`, leave all parameters the same, and click `Create training job` at the bottom of the page.

These steps will run a new version of the training job, resulting in a new model. Although we haven't materially changed the model, it provides two distinct versions which allows us to demonstrate A/B testing.

## Define a new endpoint configuration with two variants

Next, create a new endpoint configuration to demonstrate the use of A/B testing.

1. Go to `Endpoint configurations` in the SageMaker console, and locate the xgboost endpoint configuration from your earlier lab.

2. Click on that configuration.

3. Clone the configuration and rename it to `ab-test-xgboost-<your initials here>`.

4. Edit the existing model variant. Rename it to `model-v1` and give it a weight of `0.90`.

5. Add another model variant based on the second version of the xgboost model. This is the one that resulted from your second xgboost training job discussed earlier in this notebook. Rename it to `model-v2` and give it a weight of `0.10`.

6. Finally, click `Create endpoint configuration`.

## Create a new endpoint using the new endpoint configuration

Now create a new endpoint based on the new multi-variant endpoint configuration.

1. Navigate to the `Endpoints` section of the SageMaker console and click `Create endpoint`.

2. Name it `ab-test-xgboost-<your initials here>`, and choose the endpoint configuration that you just created.

3. Click `Create endpoint`.
 
4. Wait until the endpoint status moves to `In service`.

## Attach to the endpoint

In [ ]:
from sagemaker import RealTimePredictor
xgb_predictor = RealTimePredictor(endpoint='ab-test-xgboost')
    
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = csv_serializer
xgb_predictor.deserializer = None

## Run a set of predictions

In [ ]:
a = np.zeros(shape=(1,70))

In [ ]:
for i in range(5000):
    predictions = predict(a[:, 1:])

## View the invocation metrics to ensure A/B testing worked

Now you can confirm that SageMaker successfully split your traffic across the two model versions.

1. Navigate to the `Endpoints` part of the SageMaker console.
2. Click on your endpoint to get to the details page.
3. Click on `Invocation metrics` to open the CloudWatch invocation metrics for the endpoint.
4. Select the `Invocations` metric for each model variant. There should be separate metrics for `model-v1` and `model-v2`.
5. Click on the `Graphed metrics` tab and change the `Statistic` to `Sum` and the `Period` to `1 minute`.

You should see in the chart that 90% of the traffic went to `model-v1` and 10% went to `model-v2` as expected.

## Clean up

In [ ]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)